In [6]:
!nvidia-smi

Mon Jan 29 00:38:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080        Off | 00000000:01:00.0  On |                  N/A |
|  0%   47C    P8              27W / 260W |   3394MiB /  8192MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.6 🚀 Python-3.11.6 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7972MiB)
Setup complete ✅ (16 CPUs, 30.5 GB RAM, 783.9/1799.2 GB disk)


In [3]:
model = ultralytics.YOLO('yolov8m.pt')

In [4]:
results = model.val(data='coco.yaml')
results = model('https://ultralytics.com/images/bus.jpg', save=True)  # predict on an image

Ultralytics YOLOv8.1.6 🚀 Python-3.11.6 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7972MiB)
YOLOv8m summary (fused): 218 layers, 25886080 parameters, 0 gradients, 78.9 GFLOPs


val: Scanning /home/hugo/git/frc-2024-ml-note-detector/datasets/coco/labels/val2017.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [02:32<00:00,  2.05it/s]


                   all       5000      36335      0.716       0.61      0.667      0.501
                person       5000      10777      0.821      0.745      0.829      0.616
               bicycle       5000        314      0.742      0.525      0.626      0.402
                   car       5000       1918      0.765      0.637      0.713      0.497
            motorcycle       5000        367      0.805      0.677      0.792      0.548
              airplane       5000        143       0.84      0.884      0.925      0.776
                   bus       5000        283       0.85      0.784      0.864       0.75
                 train       5000        190       0.88      0.888      0.938      0.769
                 truck       5000        414      0.656       0.51      0.604      0.442
                  boat       5000        424      0.731      0.469      0.572      0.332
         traffic light       5000        634      0.723      0.483      0.569      0.318
          fire hydran

In [ ]:
model.train(data='coco128.yaml', epochs=500, batch=-1)

In [ ]:
results = model.val(data='coco.yaml')
results = model('https://ultralytics.com/images/bus.jpg', save=True)  # predict on an image



In [1]:
import ultralytics
ultralytics.checks()

model = ultralytics.YOLO('runs/detect/train/weights/best.pt')

Ultralytics YOLOv8.1.6 🚀 Python-3.11.6 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7972MiB)
Setup complete ✅ (16 CPUs, 30.5 GB RAM, 791.0/1799.2 GB disk)


In [9]:
results = model.train(data='coco.yaml', epochs=4, batch=-1)


New https://pypi.org/project/ultralytics/8.1.7 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.6 🚀 Python-3.11.6 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7972MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/train/weights/best.pt, data=coco.yaml, epochs=4, time=None, patience=50, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sho

train: Scanning /home/hugo/git/frc-2024-ml-note-detector/datasets/coco/labels/train2017.cache... 117266 images, 1021 backgrounds, 0 corrupt: 100%|██████████| 118287/118287 [00:00<?, ?it/s]
val: Scanning /home/hugo/git/frc-2024-ml-note-detector/datasets/coco/labels/val2017.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]


Plotting labels to runs/detect/train8/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train8
Starting training for 4 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/4      2.53G       1.17      1.796      1.319         25        640: 100%|██████████| 59144/59144 [1:19:11<00:00, 12.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1250/1250 [00:53<00:00, 23.35it/s]


                   all       5000      36335      0.622      0.514      0.551      0.399

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       5000      36335      0.662      0.546      0.593      0.435

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        4/4      2.51G     0.9537      1.175      1.144          1        640: 100%|██████████| 59144/59144 [1:12:03<00:00, 13.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1250/1250 [00:49<00:00, 25.47it/s]


                   all       5000      36335      0.663      0.555      0.599      0.441

4 epochs completed in 5.111 hours.
Optimizer stripped from runs/detect/train8/weights/last.pt, 52.1MB
Optimizer stripped from runs/detect/train8/weights/best.pt, 52.1MB

Validating runs/detect/train8/weights/best.pt...
Ultralytics YOLOv8.1.6 🚀 Python-3.11.6 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7972MiB)
Model summary (fused): 218 layers, 25886080 parameters, 0 gradients, 78.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1250/1250 [00:46<00:00, 26.79it/s]


                   all       5000      36335      0.664      0.554      0.599      0.441
                person       5000      10777      0.854      0.676        0.8       0.58
               bicycle       5000        314      0.643      0.471      0.524      0.324
                   car       5000       1918      0.733      0.559       0.65      0.443
            motorcycle       5000        367      0.748      0.671      0.744      0.491
              airplane       5000        143       0.81      0.804      0.867      0.701
                   bus       5000        283      0.836      0.723      0.822      0.707
                 train       5000        190      0.857      0.816      0.887      0.712
                 truck       5000        414      0.634      0.464      0.545      0.389
                  boat       5000        424      0.597      0.377      0.448      0.265
         traffic light       5000        634       0.66      0.389       0.46      0.245
          fire hydran

In [10]:
results = model.val(data='coco.yaml')
results = model('https://ultralytics.com/images/bus.jpg', save=True)  # predict on an image


Ultralytics YOLOv8.1.6 🚀 Python-3.11.6 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7972MiB)
Model summary (fused): 218 layers, 25886080 parameters, 0 gradients, 78.9 GFLOPs


val: Scanning /home/hugo/git/frc-2024-ml-note-detector/datasets/coco/labels/val2017.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2500/2500 [01:22<00:00, 30.16it/s]


                   all       5000      36335      0.663      0.555      0.599      0.441
                person       5000      10777      0.853      0.677        0.8       0.58
               bicycle       5000        314      0.641      0.468      0.523      0.323
                   car       5000       1918      0.732      0.562      0.653      0.444
            motorcycle       5000        367      0.747      0.676      0.744      0.492
              airplane       5000        143       0.81      0.804      0.867      0.701
                   bus       5000        283      0.837      0.724      0.822      0.705
                 train       5000        190      0.856      0.816      0.888      0.714
                 truck       5000        414      0.635       0.47      0.545       0.39
                  boat       5000        424      0.596       0.38      0.447      0.267
         traffic light       5000        634      0.659      0.391      0.461      0.245
          fire hydran